In [23]:
import cv2
import numpy as np
import os
import sys
from tqdm import tqdm
import random
import time

In [24]:
def down_and_up(img, down_inter_method = cv2.INTER_AREA, up_inter_method = cv2.INTER_CUBIC, max_size = 640):
    ori_h, ori_w = img.shape[:2]
    r = max_size / max(ori_w, ori_h)
    down_h, down_w = int(r * ori_h), int(r * ori_w)
    start = time.time()
    down = cv2.resize(img, (down_w, down_h), interpolation = down_inter_method)
    end = time.time()
    take = end - start
    up = cv2.resize(down, (ori_w, ori_h), interpolation = up_inter_method)
    new_h, new_w = up.shape[:2]
    assert ori_h == new_h, ori_w == new_w
    psnr = cv2.PSNR(img, up)
    return psnr, take


def test_on_imgs(imgs, down_inter_method = cv2.INTER_AREA, up_inter_method = cv2.INTER_CUBIC, max_size = 416):
    takes = []
    psnrs = []
    for i, img in tqdm(enumerate(imgs)):
        psnr, take = down_and_up(img, down_inter_method = down_inter_method, up_inter_method=up_inter_method, max_size = max_size)
        takes.append(take)
        psnrs.append(psnr)
    psnrs = np.array(psnrs)
    takes = np.array(takes)
    return psnrs.mean(), psnrs.std(), takes.mean(), takes.std()


In [25]:
TEST_DIR = '../datasets/roi_det/v2/yolox_coco/train2017/'

MAX_SIZE = 640

DOWN_METHODS = [
                'INTER_LINEAR',
                'INTER_CUBIC',
                'INTER_NEAREST',
                'INTER_LANCZOS4',
                'INTER_LINEAR_EXACT',
                'INTER_NEAREST_EXACT',
                'INTER_BITS',
                'INTER_BITS2',
    'INTER_AREA',
#                 'INTER_MAX',
#                 'INTER_TAB_SIZE',
#                 'INTER_TAB_SIZE2',
]

UP_METHODS = [
                'INTER_LINEAR',
                'INTER_CUBIC',
                'INTER_NEAREST',
                'INTER_LANCZOS4',
                'INTER_LINEAR_EXACT',
                'INTER_NEAREST_EXACT',
                'INTER_BITS',
                'INTER_BITS2',
                    'INTER_AREA',
#                 'INTER_MAX',
#                 'INTER_TAB_SIZE',
#                 'INTER_TAB_SIZE2',
]


In [4]:
img_names = os.listdir(TEST_DIR)
img_paths = [os.path.join(TEST_DIR, name) for name in img_names]
random.shuffle(img_paths)
img_paths = img_paths[:50]

imgs = []
for img_path in tqdm(img_paths):
    img = cv2.imread(img_path)
    imgs.append(img)

100%|████████████████████████████████████████████████████| 50/50 [00:09<00:00,  5.30it/s]


In [26]:
ret = {}
for down_method_name in DOWN_METHODS:
    for up_method_name in UP_METHODS:
        print(down_method_name, up_method_name)
        down_method = getattr(cv2, down_method_name)
        up_method = getattr(cv2, up_method_name)
        psnr_mean, psnr_std, time_mean, time_std = test_on_imgs(imgs, down_inter_method = down_method, up_inter_method=up_method, max_size = MAX_SIZE)
        item = {
            'psnr_mean': psnr_mean,
            'psnr_std': psnr_std,
            'time_mean': time_mean,
            'time_std': time_std
        }
        print(item)
        ret.setdefault(down_method_name, {})[up_method_name] = item

INTER_LINEAR INTER_LINEAR


50it [00:00, 62.62it/s]


{'psnr_mean': 35.166624614101764, 'psnr_std': 5.550537251550865, 'time_mean': 0.0010922861099243164, 'time_std': 0.00031551569944569493}
INTER_LINEAR INTER_CUBIC


50it [00:00, 60.85it/s]


{'psnr_mean': 35.08517061281277, 'psnr_std': 5.6512654292699755, 'time_mean': 0.001273179054260254, 'time_std': 0.0010202011704599273}
INTER_LINEAR INTER_NEAREST


...

{'psnr_mean': 35.47054629098174, 'psnr_std': 5.46088732088183, 'time_mean': 0.029208674430847167, 'time_std': 0.008979191254161599}
INTER_AREA INTER_BITS2


50it [00:02, 22.52it/s]


{'psnr_mean': 36.07677906808241, 'psnr_std': 5.5678416595541576, 'time_mean': 0.028345279693603516, 'time_std': 0.008786037531763765}
INTER_AREA INTER_AREA


50it [00:02, 21.67it/s]

{'psnr_mean': 34.854101032798866, 'psnr_std': 4.981452770051216, 'time_mean': 0.02906826972961426, 'time_std': 0.008864168183807026}


In [27]:
cv2.PSNR(img, img)

361.20199909921956

In [28]:
cv2.PSNR(np.ones((2000, 2000)), np.ones((2000, 2000)))

361.20199909921956

In [29]:
cv2.PSNR(np.zeros((2000, 2000)), np.zeros((2000, 2000)))

361.20199909921956

In [30]:
d = {}
for down_method_name, v1 in ret.items():
    temp = [(k,v) for k, v in v1.items()]
    temp.sort(key = lambda x: x[1]['psnr_mean'], reverse = True)
    d[down_method_name] = temp[0]

### Results
- AREA:
    + 416: 47 +- 28 ms
    + 640: 29 +- 9
- LINEAR
    + 416: 1.2 +-0.8 ms
    + 640: 1.3 +-0.8 ms
- LINEAR EXACT:
    + 416: 1.9 +- 0.6 ms
    + 640: 2.2 +-0.5 ms
- CUBIC
    + 416: 0.8 +- 0.5 ms
    + 640: 1.9 +- 1.0 ms

In [31]:
d2 = [(k, v[0], v[1]) for k, v in d.items()]
d2.sort(key = lambda x: x[2]['psnr_mean'], reverse = True)
d2

[('INTER_AREA',
  'INTER_LANCZOS4',
  {'psnr_mean': 36.156532855081146,
   'psnr_std': 5.578780077261077,
   'time_mean': 0.029025239944458006,
   'time_std': 0.009666186039810667}),
 ('INTER_LINEAR_EXACT',
  'INTER_LINEAR_EXACT',
  {'psnr_mean': 35.18274839001916,
   'psnr_std': 5.578821294615943,
   'time_mean': 0.002215018272399902,
   'time_std': 0.0005492360048967448}),
 ('INTER_BITS',
  'INTER_LINEAR_EXACT',
  {'psnr_mean': 35.18274839001916,
   'psnr_std': 5.578821294615943,
   'time_mean': 0.002053065299987793,
   'time_std': 0.00036337908583282554}),
 ('INTER_LINEAR',
  'INTER_LINEAR_EXACT',
  {'psnr_mean': 35.17824584755071,
   'psnr_std': 5.573475322722324,
   'time_mean': 0.0013345813751220704,
   'time_std': 0.0008717576946383272}),
 ('INTER_CUBIC',
  'INTER_LINEAR_EXACT',
  {'psnr_mean': 34.92353300669888,
   'psnr_std': 5.607717233316001,
   'time_mean': 0.0019814205169677735,
   'time_std': 0.001095720139731244}),
 ('INTER_BITS2',
  'INTER_LINEAR_EXACT',
  {'psnr_mean':

In [32]:
ret['INTER_AREA']

{'INTER_LINEAR': {'psnr_mean': 35.46514054349144,
  'psnr_std': 5.450918063657956,
  'time_mean': 0.028186535835266112,
  'time_std': 0.009063185792128947},
 'INTER_CUBIC': {'psnr_mean': 36.07677906808241,
  'psnr_std': 5.5678416595541576,
  'time_mean': 0.028650541305541993,
  'time_std': 0.009735113960147325},
 'INTER_NEAREST': {'psnr_mean': 34.24488380777067,
  'psnr_std': 4.564733033916409,
  'time_mean': 0.027912707328796388,
  'time_std': 0.008723845932457798},
 'INTER_LANCZOS4': {'psnr_mean': 36.156532855081146,
  'psnr_std': 5.578780077261077,
  'time_mean': 0.029025239944458006,
  'time_std': 0.009666186039810667},
 'INTER_LINEAR_EXACT': {'psnr_mean': 35.47054629098174,
  'psnr_std': 5.46088732088183,
  'time_mean': 0.030905332565307617,
  'time_std': 0.009579332900098523},
 'INTER_NEAREST_EXACT': {'psnr_mean': 34.41073997543038,
  'psnr_std': 4.697880057738437,
  'time_mean': 0.030565156936645507,
  'time_std': 0.01006303160473994},
 'INTER_BITS': {'psnr_mean': 35.47054629098